# \begin{equation*} \frac{D \sigma}{D t} = \dot\sigma_{\theta} = \frac{\partial \sigma_{\theta}}{\partial \theta} \dot\theta + \frac{\partial \sigma_{\theta}}{\partial S} \dot S \end{equation*}
## **SOSE**

In [1]:
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
cluster.scale(30)
#cluster.adapt(minimum=2, maximum=25)  # or cluster.scale(n) to a fixed size.
client = cluster.get_client()

In [2]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [29]:
client.close()
cluster.close()

In [3]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import fsspec
%matplotlib inline
import fastjmd95.jmd95numba as jmd95numba
#from fastjmd95 import jmd95numba
#from fastjmd95 import rho, drhods, drhodt
import dask.array as dsa

In [4]:
xr.set_options(display_style='text')

In [5]:
fs = gcsfs.GCSFileSystem(requester_pays=True)

mapping = fs.get_mapper('gcs://pangeo-ecco-sose')

ds = xr.open_zarr(mapping, consolidated=True)
ds

<xarray.Dataset>
Dimensions:   (XC: 2160, XG: 2160, YC: 320, YG: 320, Z: 42, Zl: 42, Zp1: 43, Zu: 42, time: 438)
Coordinates:
    Depth     (YC, XC) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    PHrefC    (Z) float32 dask.array<chunksize=(42,), meta=np.ndarray>
    PHrefF    (Zp1) float32 dask.array<chunksize=(43,), meta=np.ndarray>
  * XC        (XC) float32 0.083333336 0.25 0.4166667 ... 359.75 359.9167
  * XG        (XG) float32 5.551115e-17 0.16666667 ... 359.6667 359.83334
  * YC        (YC) float32 -77.87497 -77.7083 -77.54163 ... -24.874966 -24.7083
  * YG        (YG) float32 -77.9583 -77.79163 -77.62497 ... -24.9583 -24.791632
  * Z         (Z) float32 -5.0 -15.5 -27.0 -39.5 ... -5075.0 -5325.0 -5575.0
  * Zl        (Zl) float32 0.0 -10.0 -21.0 -33.0 ... -4950.0 -5200.0 -5450.0
  * Zp1       (Zp1) float32 0.0 -10.0 -21.0 -33.0 ... -5200.0 -5450.0 -5700.0
  * Zu        (Zu) float32 -10.0 -21.0 -33.0 -46.0 ... -5200.0 -5450.0 -5700.0
    drC       (Zp1) float32 dask.array<chunksize=(43,), meta=np.ndarray>
    drF       (Z) float32 dask.array<chunksize=(42,), meta=np.ndarray>
    dxC       (YC, XG) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    dxG       (YG, XC) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    dyC       (YG, XC) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    dyG       (YC, XG) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    hFacC     (Z, YC, XC) float32 dask.array<chunksize=(42, 320, 2160), meta=np.ndarray>
    hFacS     (Z, YG, XC) float32 dask.array<chunksize=(42, 320, 2160), meta=np.ndarray>
    hFacW     (Z, YC, XG) float32 dask.array<chunksize=(42, 320, 2160), meta=np.ndarray>
    iter      (time) int64 dask.array<chunksize=(438,), meta=np.ndarray>
    rA        (YC, XC) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    rAs       (YG, XC) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    rAw       (YC, XG) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
    rAz       (YG, XG) float32 dask.array<chunksize=(320, 2160), meta=np.ndarray>
  * time      (time) datetime64[ns] 2005-01-06 2005-01-11 ... 2010-12-31
Data variables:
    ADVr_SLT  (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    ADVr_TH   (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    ADVx_SLT  (time, Z, YC, XG) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    ADVx_TH   (time, Z, YC, XG) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    ADVy_SLT  (time, Z, YG, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    ADVy_TH   (time, Z, YG, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFrE_SLT  (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFrE_TH   (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFrI_SLT  (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFrI_TH   (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFxE_SLT  (time, Z, YC, XG) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFxE_TH   (time, Z, YC, XG) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFyE_SLT  (time, Z, YG, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DFyE_TH   (time, Z, YG, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    DRHODR    (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    ETAN      (time, YC, XC) float32 dask.array<chunksize=(1, 320, 2160), meta=np.ndarray>
    EXFswnet  (time, YC, XC) float32 dask.array<chunksize=(1, 320, 2160), meta=np.ndarray>
    KPPg_SLT  (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 42, 320, 2160), meta=np.ndarray>
    KPPg_TH   (time, Zl, YC, XC) float32 d

In [6]:
pref = 2000

# Matrix of possible computing options
### 12 in total, but for last 4 couldn't figure out how to run with `xr.map_blocks()`

In [37]:
ds.THETA.data

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,439 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [7]:
#Randomized data
shape = (438, 42, 320, 2160)
chunks = (1, 42, 320, 2160)
#salt = dsa.full(shape, 35., chunks=chunks, dtype='f4')
#theta = dsa.full(shape, 10., chunks=chunks, dtype='f4')
salt = dsa.random.random(shape, chunks=chunks).astype('f4')
theta = dsa.random.random(shape, chunks=chunks).astype('f4')
pref = 2000
salt

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,876 Tasks,438 Chunks
Type,float32,numpy.ndarray


### 1. **`xr.apply_ufunc()`-`fastjmd95`-model data**

In [8]:
drhodt_ufunc = xr.apply_ufunc(jmd95numba.drhodt, ds.SALT, ds.THETA, pref,
                              output_dtypes=[ds.THETA.dtype],
                              dask='parallelized').reset_coords(drop=True)
drhodt_ufunc.data

,Array,Chunk
Bytes,101.72 GB,232.24 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,2193 Tasks,438 Chunks
Type,float64,numpy.ndarray


In [9]:
%%time
drhodt_ufunc.mean(dim=('time', 'Z')).compute()

CPU times: user 1.56 s, sys: 68.4 ms, total: 1.63 s
Wall time: 4min 4s


<xarray.DataArray (YC: 320, XC: 2160)>
array([[-0.00478695, -0.00478695, -0.00478695, ..., -0.00478695,
        -0.00478695, -0.00478695],
       [-0.00478695, -0.00478695, -0.00478695, ..., -0.00478695,
        -0.00478695, -0.00478695],
       [-0.00478695, -0.00478695, -0.00478695, ..., -0.00478695,
        -0.00478695, -0.00478695],
       ...,
       [-0.1917631 , -0.19167165, -0.19157862, ..., -0.18894047,
        -0.18885288, -0.18876377],
       [-0.19187875, -0.19178671, -0.19169259, ..., -0.19354937,
        -0.19346012, -0.18887939],
       [-0.19151946, -0.19142568, -0.19133256, ..., -0.18870878,
        -0.18861612, -0.18852244]])
Coordinates:
  * XC       (XC) float32 0.083333336 0.25 0.4166667 ... 359.75 359.9167
  * YC       (YC) float32 -77.87497 -77.7083 -77.54163 ... -24.874966 -24.7083

### 2. **`xr.apply_ufunc()`-`fastjmd95`-randomized data**

In [10]:
salt

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,876 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [11]:
dummy_ds = xr.Dataset({'theta':(['time', 'Z','YC','XC'], theta),
                       'salt': (['time', 'Z','YC','XC'], salt)
                      })

In [12]:
dummy_ds.salt

<xarray.DataArray 'salt' (time: 438, Z: 42, YC: 320, XC: 2160)>
dask.array<astype, shape=(438, 42, 320, 2160), dtype=float32, chunksize=(1, 42, 320, 2160), chunktype=numpy.ndarray>
Dimensions without coordinates: time, Z, YC, XC

In [14]:
drhodt_ufunc_random = xr.apply_ufunc(jmd95numba.drhodt, dummy_ds.salt, dummy_ds.theta, pref,
                              output_dtypes=[dummy_ds.theta.dtype],
                              dask='parallelized').reset_coords(drop=True)
drhodt_ufunc_random.data

,Array,Chunk
Bytes,101.72 GB,232.24 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,3067 Tasks,438 Chunks
Type,float64,numpy.ndarray


In [16]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


### Cell below: all tasks go to one worker and it never executes

In [17]:
%%time
drhodt_ufunc_random.mean(dim=('time', 'Z')).compute()

KeyboardInterrupt: 

### 3. **`xr.apply_ufunc()`-`sum()`-model data**

In [18]:
def dummy_function(a, b, c):
    return a + b + c

dummy_ufunc = xr.apply_ufunc(dummy_function, ds.SALT, ds.THETA, pref,
                        output_dtypes=[ds.THETA.dtype],
                        dask='parallelized').reset_coords(drop=True)

In [19]:
dummy_ufunc.data

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,2193 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [20]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [21]:
%time dummy_ufunc.mean(dim=('time', 'Z')).compute()

CPU times: user 342 ms, sys: 23.5 ms, total: 365 ms
Wall time: 29.7 s


<xarray.DataArray (YC: 320, XC: 2160)>
array([[2000.    , 2000.    , 2000.    , ..., 2000.    , 2000.    ,
        2000.    ],
       [2000.    , 2000.    , 2000.    , ..., 2000.    , 2000.    ,
        2000.    ],
       [2000.    , 2000.    , 2000.    , ..., 2000.    , 2000.    ,
        2000.    ],
       ...,
       [2042.6941, 2042.6805, 2042.6671, ..., 2041.8556, 2041.8425,
        2041.8295],
       [2042.7092, 2042.6958, 2042.6819, ..., 2041.8706, 2041.8578,
        2041.8447],
       [2042.6627, 2042.6493, 2042.6357, ..., 2041.8256, 2041.8121,
        2041.7987]], dtype=float32)
Coordinates:
  * XC       (XC) float32 0.083333336 0.25 0.4166667 ... 359.75 359.9167
  * YC       (YC) float32 -77.87497 -77.7083 -77.54163 ... -24.874966 -24.7083

### 4. **`xr.apply_ufunc()`-`sum()`-randomized data**

In [22]:
dummy_ufunc_random = xr.apply_ufunc(dummy_function, dummy_ds.salt, dummy_ds.theta, pref,
                        output_dtypes=[dummy_ds.theta.dtype],
                        dask='parallelized').reset_coords(drop=True)

In [23]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [24]:
%time dummy_ufunc_random.mean(dim=('time', 'Z')).compute()

CPU times: user 523 ms, sys: 40.9 ms, total: 564 ms
Wall time: 15.6 s


<xarray.DataArray (YC: 320, XC: 2160)>
array([[2001.005 , 2000.9963, 2001.0015, ..., 2000.9985, 2000.9965,
        2001.0028],
       [2001.0002, 2001.0011, 2000.9894, ..., 2001.0024, 2001.0028,
        2001.0009],
       [2000.9976, 2000.9994, 2000.9998, ..., 2001.0045, 2000.9994,
        2001.0002],
       ...,
       [2000.9958, 2001.0022, 2001.0028, ..., 2000.9974, 2001.0011,
        2001.0028],
       [2000.995 , 2000.9994, 2000.9961, ..., 2000.998 , 2000.9926,
        2000.9976],
       [2000.9974, 2001.002 , 2001.0006, ..., 2001.0011, 2000.9989,
        2000.9998]], dtype=float32)
Dimensions without coordinates: YC, XC

### 5. **`dsa.map_blocks()`-`fastjmd95`-model data**

In [25]:
drhodt_dsa_mapped = dsa.map_blocks(
         jmd95numba.drhodt, ds.SALT.data, ds.THETA.data, pref,
         dtype=ds.SALT.dtype)
drhodt_dsa_mapped

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,1316 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [26]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [27]:
%time drhodt_dsa_mapped.mean(axis=(0, 1,)).compute()

CPU times: user 211 ms, sys: 12.2 ms, total: 223 ms
Wall time: 51.2 s


array([[-0.00478695, -0.00478695, -0.00478695, ..., -0.00478695,
        -0.00478695, -0.00478695],
       [-0.00478695, -0.00478695, -0.00478695, ..., -0.00478695,
        -0.00478695, -0.00478695],
       [-0.00478695, -0.00478695, -0.00478695, ..., -0.00478695,
        -0.00478695, -0.00478695],
       ...,
       [-0.19176309, -0.19167164, -0.19157861, ..., -0.18894047,
        -0.18885289, -0.18876377],
       [-0.19187874, -0.1917867 , -0.19169256, ...,         nan,
                nan, -0.18887939],
       [-0.19151944, -0.19142567, -0.19133256, ..., -0.18870878,
        -0.18861613, -0.18852244]], dtype=float32)

### 6. **`dsa.map_blocks()`-`fastjmd95`-randomized data**

In [29]:
drhodt_dsa_mapped_random = dsa.map_blocks(
         jmd95numba.drhodt, salt, theta, pref,
         dtype=salt.dtype)
drhodt_dsa_mapped_random

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,2190 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [28]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [30]:
%time drhodt_dsa_mapped_random.mean(axis=(0, 1,)).compute()

CPU times: user 1.47 s, sys: 34.3 ms, total: 1.5 s
Wall time: 1min 53s


array([[-0.01428219, -0.01421242, -0.01424168, ..., -0.01422201,
        -0.0141891 , -0.01424628],
       [-0.01424688, -0.01418688, -0.01413411, ..., -0.01424772,
        -0.01423902, -0.01423039],
       [-0.01421119, -0.01423629, -0.01420496, ..., -0.01424887,
        -0.01419087, -0.01421663],
       ...,
       [-0.01419654, -0.01425527, -0.01425948, ..., -0.01418888,
        -0.01424244, -0.0142272 ],
       [-0.01418868, -0.01425183, -0.0141712 , ..., -0.01417595,
        -0.01416688, -0.01419552],
       [-0.01421727, -0.01423457, -0.01425139, ..., -0.01424719,
        -0.01420887, -0.01422001]], dtype=float32)

### 7. **`dsa.map_blocks()`-`sum()`-model data**

In [32]:
dummy_dsa_mapped = dsa.map_blocks(
    dummy_function, ds.SALT.data, ds.THETA.data, pref,
    dtype=ds.THETA.dtype)
dummy_dsa_mapped

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,1316 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [31]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [33]:
%time dummy_dsa_mapped.mean(axis=(0, 1)).compute()

CPU times: user 195 ms, sys: 12 ms, total: 207 ms
Wall time: 27.7 s


array([[2000.    , 2000.    , 2000.    , ..., 2000.    , 2000.    ,
        2000.    ],
       [2000.    , 2000.    , 2000.    , ..., 2000.    , 2000.    ,
        2000.    ],
       [2000.    , 2000.    , 2000.    , ..., 2000.    , 2000.    ,
        2000.    ],
       ...,
       [2042.6941, 2042.6805, 2042.6671, ..., 2041.8556, 2041.8425,
        2041.8295],
       [2042.7092, 2042.6958, 2042.6819, ..., 2041.8706, 2041.8578,
        2041.8447],
       [2042.6627, 2042.6493, 2042.6357, ..., 2041.8256, 2041.8121,
        2041.7987]], dtype=float32)

### 8. **`dsa.map_blocks()`-`sum()`-randomized data**

In [35]:
dummy_dsa_mapped_random = dsa.map_blocks(
                          dummy_function, salt, theta, pref,
                          dtype=theta.dtype)
dummy_dsa_mapped_random

,Array,Chunk
Bytes,50.86 GB,116.12 MB
Shape,"(438, 42, 320, 2160)","(1, 42, 320, 2160)"
Count,2190 Tasks,438 Chunks
Type,float32,numpy.ndarray


In [34]:
client.restart()

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.2e6facec3e1a4491ae4001887caf03ba Dashboard: /services/dask-gateway/clusters/prod.2e6facec3e1a4491ae4001887caf03ba/status,Cluster Workers: 30 Cores: 60 Memory: 257.70 GB


In [36]:
%time dummy_dsa_mapped_random.mean(axis=(0,1)).compute()

CPU times: user 336 ms, sys: 21 ms, total: 357 ms
Wall time: 13.3 s


array([[2001.005 , 2000.9963, 2001.0015, ..., 2000.9985, 2000.9965,
        2001.0028],
       [2001.0002, 2001.0011, 2000.9894, ..., 2001.0024, 2001.0028,
        2001.0009],
       [2000.9976, 2000.9994, 2000.9998, ..., 2001.0045, 2000.9994,
        2001.0002],
       ...,
       [2000.9958, 2001.0022, 2001.0028, ..., 2000.9974, 2001.0011,
        2001.0028],
       [2000.995 , 2000.9994, 2000.9961, ..., 2000.998 , 2000.9926,
        2000.9976],
       [2000.9974, 2001.002 , 2001.0006, ..., 2001.0011, 2000.9989,
        2000.9998]], dtype=float32)

### 9. **`xr.map_blocks()`-`fastjmd95`-model data**

In [ ]:
drhodt_xr_mapped = ds.map_blocks(
                   jmd95numba.drhodt, 
                   args=[ds.SALT, ds.THETA, pref], 
                   template=ds)
drhodt_xr_mapped

In [ ]:
%time drhodt_xr_mapped.THETA.mean(('time', 'Z')).compute()

### 10. **`xr.map_blocks()`-`fastjmd95`-randomized data**

In [ ]:
drhodt_xr_mapped_random = xr.map_blocks(
         jmd95numba.drhodt, salt, theta, pref,
         dtype=salt.dtype)
drhodt_xr_random

### 11. **`xr.map_blocks()`-`sum()`-model data**

In [ ]:
dummy_xr_mapped = xr.map_blocks(
                   jmd95numba.drhodt, ds.SALT, ds.THETA, pref,
                   dtype=ds.SALT.dtype)
dummy_xr_mapped

### 12. **`xr.map_blocks()`-`sum()`-randomized data**

In [ ]:
dummy_xr_mapped_random = xr.map_blocks(
                   dummy_function, salt, theta, pref,
                   dtype=salt.dtype)
dummy_xr_mapped_random